#### BA2A: Implement MotifEnumeration

In [5]:
def generateAllKmers(k):
    base_neucleotides = ["A", "T", "G", "C"]
    last_base = base_neucleotides #copying base neucleotides in last_base
    current_array = [] #declare an emty list
    for i in range(k-1): 
        for j in base_neucleotides: #A, T, G, C
            for k in last_base: #copy of A, T, G, C
                current_array.append(j+k) 
        last_base = current_array #update last_base
        current_array = [] #empty current array
    return last_base

def MismatchedMotifSet(text, k, d):
    motifset = set()
    all_Kmer = generateAllKmers(k) #generate all possible k mers
    text_Kmer = textKmer(text, k)  #generate all possible kmers from given text
    for i in range(len(all_Kmer)):
        for j in range(len(text_Kmer)):
          if hammingDistance(all_Kmer[i], text_Kmer[j]) <= d: #for each kmer generated from all possible kmers check hamming distance with main text kmers
            motifset.add(all_Kmer[i]) 
    return motifset
    
def hammingDistance(text1, text2):
    count=0;
    for i in range(len(text1)):
        if(text1[i] != text2[i]):
            count+=1
    return count

def textKmer(text, k):
    my_arr = []
    for i in range(len(text)-k+1):
        my_arr.append(text[i:i+k])
    return my_arr

def DNeighbourhood(genomelist, k,d):
  count = {}

  for genome in genomelist:
    motifset = MismatchedMotifSet(genome, k, d)
    for motif in motifset:
      if motif not in count:
        count[motif] = 1
      else:
        count[motif] += 1
  
  motiflist = []
  for key in count:
    if count[key] == len(genomelist):
      motiflist.append(key)
  return sorted(motiflist)

def WriteToFile(result):
  outF = open("myOutFile.txt", "w")
  for line in result:
    # write line to output file
    outF.write(line)
    outF.write(" ")
  outF.close()

k = 3
d = 1
with open('/content/rosalind_ba2a.txt', 'r') as f:
  genomelist = [line.strip() for line in f.readlines()]
length = len(genomelist)
ans = DNeighbourhood(genomelist, k, d)
print(ans)


['ATA', 'ATT', 'GTT', 'TTT']


#### BA2B: Find a Median String

In [7]:
neucleotide_base = {0:'A', 1:'C', 2:'G', 3:'T'}
def NumberToPattern(index, k):
    if(k == 1): #base case
        return neucleotide_base[index]
    pref_index = int(index/4)
    rem = int(index%4)
    symbol = neucleotide_base[rem] #remainder is the single symbol
    pref_pattern = NumberToPattern(pref_index, k-1) #recursive call
    
    return pref_pattern+symbol

def generateTextKmer(text, k):
  kmers = set()
  for i in range(len(text)-k+1):
    kmers.add(text[i:i+k])
  return kmers

def HammingDistance(text1, text2):
    count=0;
    for i in range(len(text1)):
        if(text1[i] != text2[i]):
            count+=1
    return count

def DistanceBetweenPatternsandGenome(genomelist, pattern):
  k = len(pattern)
  distance = 0
  for genome in genomelist:
    hammingDistance = 9999
    genome_kmers = generateTextKmer(genome, k)
    for kmer in  genome_kmers:
      if hammingDistance > HammingDistance(pattern, kmer):
        hammingDistance =  HammingDistance(pattern, kmer)
    distance = distance+hammingDistance
  #print(distance)  
  return distance

def MedianString(genomelist, k):
  distance = 9999;
  limit = pow(4,k)
  for i in range(limit-1):
    pattern = NumberToPattern(i,k)
    if distance >= DistanceBetweenPatternsandGenome(genomelist, pattern):
      #print("pattern, Distance")
      #print(pattern, distance)
      distance = DistanceBetweenPatternsandGenome(genomelist, pattern)
      median = pattern
  return median

k = int(input("Enter the length of kmer: "))
genomelist = []
while True:
  line = input().upper()
  if line:
    genomelist.append(line)
  else:
    break
#print(genomelist)
ans = MedianString(genomelist, k)
print(ans)



Enter the length of kmer: 3
AAATTGACGCAT
GACGACCACGTT
CGTCAGCGCCTG
GCTGAGCACCGG
AGTACGGGACAG

GAC


#### BA2C: Find a Profile-most Probable k-mer in a String

In [8]:
import numpy as np

def generateTextKmer(text, k):
  kmers = set()
  for i in range(len(text)-k+1):
    kmers.add(text[i:i+k])
  return kmers

def ProfileMostProbableKmer(genome, k, profile_matrix):
  kmers = generateTextKmer(genome, k)
  maxProb = -9999
  probKmer = ""
  for kmer in kmers:
    prob = 1
    for i in range(len(kmer)):
      if kmer[i] == 'A':
        prob = prob*profile_matrix[0][i]
      elif kmer[i] == 'C':
        prob = prob*profile_matrix[1][i]
      elif kmer[i] == 'G':
        prob = prob*profile_matrix[2][i]
      elif kmer[i] == 'T':
        prob = prob*profile_matrix[3][i]
    if maxProb < prob:
      maxProb = prob
      probKmer = kmer

  return probKmer


genome = input("Enter the genome: ")
k = int(input("Enter pattern length: "))
print("Enter the entries in a single line separated by space: ")
arr_entry1 = list(map(float, input().strip().split()))
profile_matrix =  np.array(arr_entry1).reshape(4,k)
ProfileMostProbableKmer(genome, k, profile_matrix)

Enter the genome: ACCTGTTTATTGCCTAAGTTCCGAACAAACCCAATATAGCCCGAGGGCCT
Enter pattern length: 5
Enter the entries in a single line separated by space: 
0.2 0.2 0.3 0.2 0.3 0.4 0.3 0.1 0.5 0.1 0.3 0.3 0.5 0.2 0.4 0.1 0.2 0.1 0.1 0.2


'CCGAG'

#### BA2D: Implement GreedyMotifSearch

In [9]:
import numpy as np

def BestMotifs(genomes, k,t):
  bestmotifs = []
  for genome in genomes:
    bestmotifs.append(genome[:k])
  return bestmotifs

def generateTextKmer(text, k):
  kmers = set()
  for i in range(len(text)-k+1):
    kmers.add(text[i:i+k])
  return kmers

def CountMatrix(motifs): 
  count = np.zeros((4,len(motifs[0])))
  for motif in motifs:
    for i in range(len(motif)):
      if motif[i-1] == 'A':
        count[0][i-1] += 1
      elif motif[i-1] == 'C':
        count[1][i-1] += 1
      elif motif[i-1] == 'G':
        count[2][i-1] += 1
      elif motif[i-1] == 'T':
        count[3][i-1] += 1
  return count

def ProfileMatrix(motifs): 
  countMatrix = CountMatrix(motifs)
  profileMatrix = np.zeros((4,len(motifs[0])))
  #divide count matrix by number of motifs
  for row in range(4):
    for col in range(k):
      profileMatrix[row][col] = countMatrix[row][col] / len(motifs)
  return profileMatrix


def ProfileMostProbableKmer(genome, k, profile_matrix):
  maxprob = 0
  kmer = genome[0:k]
  for i in range(1,len(genome) - k +1):
    prob = 1
    pattern = genome[i:i+k]
    for j in range(k):
      l = symbolToNumber(pattern[j])
      prob *= profile_matrix[l][j]
    if prob > maxprob:
      maxprob = prob
      kmer = pattern
  return kmer
  
def symbolToNumber(symbol):
	if symbol == "A":
		return 0
	if symbol == "C":
		return 1
	if symbol == "G":
		return 2
	if symbol == "T":
		return 3

def NumberToSymbol(number):
	if number == 0:
		return "A"
	if number == 1:
		return "C"
	if number == 2:
		return "G"
	if number == 3:
		return "T"

def Consensus(profile):
  consensus = ""
  for col in range(len(profile[0])):  
    max = 0
    loc = 0
    for row in range(4):
      if profile[row][col] > max:
        loc = row  
        max = profile[row][col]  
    consensus += NumberToSymbol(loc)  
  return consensus

neucleotide_base = {0:'A', 1:'C', 2:'G', 3:'T'}
def NumberToPattern(index, k):
    if(k == 1): 
        return neucleotide_base[index]
    pref_index = int(index/4)
    rem = int(index%4)
    symbol = neucleotide_base[rem] 
    pref_pattern = numberToPattern(pref_index, k-1) 
    
    return pref_pattern+symbol

def Score(motifs):
  profile = ProfileMatrix(motifs)
  cons = Consensus(profile)
  score = 0
  for motif in motifs:  
    for i in range(len(motif)): 
      if cons[i] != motif[i]:
        score+=1
  return score

def GreedyMotif(genomes, k, t):
  bestMotifs = BestMotifs(genomes, k,t)
  kmers = generateTextKmer(genomes[0], k)
  for kmer in kmers:
    motifs = []
    motifs.append(kmer)
    for idx in range(1, t):
      profile_matrix = ProfileMatrix(motifs) 
      motifs.append(ProfileMostProbableKmer(genomes[idx], k, profile_matrix))
    if Score(motifs) < Score(bestMotifs):
      bestMotifs = motifs
  return bestMotifs

k = 3
t = 5
with open('/content/rosalind_ba2d.txt', 'r') as f:
  genomelist = [line.strip() for line in f.readlines()]
ans = GreedyMotif(genomelist, k, t)
for idx in ans:
  print(idx)

CAG
CAG
CAA
CAA
CAA


#### BA2E: 
Implement GreedyMotifSearch with Pseudocounts

In [13]:
import numpy as np
def BestMotifs(genomes, k,t):
  bmotifs = []
  for genome in genomes:
    bmotifs.append(genome[:k])
  return bmotifs

def generateTextKmer(text, k):
  kmers = set()
  for i in range(len(text)-k+1):
    kmers.add(text[i:i+k])
  return kmers


def CountMatrix(motifs): # count matrix is 4*k AAGG
  count = np.zeros((4,len(motifs[0])))
  for motif in motifs:
    #print(motif)
    for i in range(len(motif)):
      # print("i: ", i)
      # print(motif[i])
      if motif[i-1] == 'A':
        count[0][i-1] += 1
      elif motif[i-1] == 'C':
        count[1][i-1] += 1
      elif motif[i-1] == 'G':
        count[2][i-1] += 1
      elif motif[i-1] == 'T':
        count[3][i-1] += 1
  return count

def ProfileMatrix(motifs): 
  countMat = CountMatrix(motifs)
  pseudoMat = PseudoCountMatrix(countMat)
  profileMatrix = np.zeros((4,len(motifs[0])))
  # now just divide count matrix by number of motifs
  for row in range(4):
    for col in range(k):
      profileMatrix[row][col] = pseudoMat[row][col] / len(motifs)
  return profileMatrix

def PseudoCountMatrix(countMat):
  pseudo = [1*4]*len(countMat[0])
  pseudoMatrix = pseudo+countMat
 #print("pseudo", pseudo)
  return pseudoMatrix


def ProfileMostProbableKmer(genome, k, profile_matrix):
  maxprob = 0
  kmer = genome[0:k]
  for i in range(1,len(genome) - k +1):
    prob = 1
    pattern = genome[i:i+k]
    for j in range(k):
      l = symbolToNumber(pattern[j])
      prob *= profile_matrix[l][j]
    if prob > maxprob:
      maxprob = prob
      kmer = pattern
  return kmer
  
def symbolToNumber(symbol):
	if symbol == "A":
		return 0
	if symbol == "C":
		return 1
	if symbol == "G":
		return 2
	if symbol == "T":
		return 3

def NumberToSymbol(number):
	if number == 0:
		return "A"
	if number == 1:
		return "C"
	if number == 2:
		return "G"
	if number == 3:
		return "T"

def Consensus(profile):
  consensus = ""
  for col in range(len(profile[0])):  #column wise
    max = 0
    loc = 0
    for row in range(4):
      if profile[row][col] > max:
        loc = row 
        max = profile[row][col]
    consensus += NumberToSymbol(loc)  
  return consensus

neucleotide_base = {0:'A', 1:'C', 2:'G', 3:'T'}
def NumberToPattern(index, k):
    if(k == 1): #base case
        return neucleotide_base[index]
    pref_index = int(index/4)
    rem = int(index%4)
    symbol = neucleotide_base[rem]
    pref_pattern = numberToPattern(pref_index, k-1) 
    
    return pref_pattern+symbol

def Score(motifs):
  profile = ProfileMatrix(motifs)
  cons = Consensus(profile)
  score = 0
  for motif in motifs:  
    for i in range(len(motif)): 
      if cons[i] != motif[i]:
        score+=1
  return score

def GreedyMotifWithPseudoCount(genomes, k, t):
  bestMotifs = BestMotifs(genomes, k,t)
  kmers = generateTextKmer(genomes[0], k)
  for kmer in kmers:
    motifs = []
    motifs.append(kmer)
    for idx in range(1, t):
      profile_matrix = ProfileMatrix(motifs) 
      motifs.append(ProfileMostProbableKmer(genomes[idx], k, profile_matrix))
    
    if Score(motifs) < Score(bestMotifs):
      bestMotifs = motifs
  return bestMotifs

k = 3
t = 5
with open('/content/rosalind_ba2e.txt', 'r') as f:
  genomelist = [line.strip() for line in f.readlines()]
ans = GreedyMotifWithPseudoCount(genomelist, k, t)
for i in ans:
  print(i)

TTC
ATC
TTC
ATC
TTC


#### BA2F: Implement RandomizedMotifSearch

In [14]:
import numpy as np
from random import randint

def generateTextKmer(text, k):
  kmers = set()
  for i in range(len(text)-k+1):
    kmers.add(text[i:i+k])
  return kmers

def CountMatrix(motifs): 
  count = np.zeros((4,len(motifs[0])))
  for motif in motifs:
    for i in range(len(motif)):
      if motif[i-1] == 'A':
        count[0][i-1] += 1
      elif motif[i-1] == 'C':
        count[1][i-1] += 1
      elif motif[i-1] == 'G':
        count[2][i-1] += 1
      elif motif[i-1] == 'T':
        count[3][i-1] += 1
  return count

def ProfileMatrix(motifs):
  countMat = CountMatrix(motifs)
  pseudoMat = PseudoCountMatrix(countMat)
  profileMatrx = np.zeros((4,len(motifs[0])))
  for row in range(4):
    for col in range(k):
      profileMatrx[row][col] = pseudoMat[row][col] / len(motifs)
  return profileMatrx

def PseudoCountMatrix(countMat):
  pseudo = [1*4]*len(countMat[0])
  pseudoMatrix = pseudo+countMat
  return pseudoMatrix

def ProfileMostProbableKmer(genome, k, profile_matrix):
  maxprob = 0
  kmer = genome[0:k]
  for i in range(1,len(genome) - k +1):
    prob = 1
    pattern = genome[i:i+k]
    for j in range(k):
      l = symbolToNumber(pattern[j])
      prob *= profile_matrix[l][j]
    if prob > maxprob:
      maxprob = prob
      kmer = pattern
  return kmer
  
def symbolToNumber(symbol):
	if symbol == "A":
		return 0
	if symbol == "C":
		return 1
	if symbol == "G":
		return 2
	if symbol == "T":
		return 3

def NumberToSymbol(number):
	if number == 0:
		return "A"
	if number == 1:
		return "C"
	if number == 2:
		return "G"
	if number == 3:
		return "T"

def Consensus(profile):
  consensus = ""
  for col in range(len(profile[0])):  
    max = 0
    loc = 0
    for row in range(4):
      if profile[row][col] > max:
        loc = row  
        max = profile[row][col]  
    consensus += NumberToSymbol(loc)  
  return consensus

neucleotide_base = {0:'A', 1:'C', 2:'G', 3:'T'}
def NumberToPattern(index, k):
    if(k == 1): #base case
        return neucleotide_base[index]
    pref_index = int(index/4)
    rem = int(index%4)
    symbol = neucleotide_base[rem] 
    pref_pattern = numberToPattern(pref_index, k-1) 
    
    return pref_pattern+symbol

def Score(motifs):
  profile = ProfileMatrix(motifs)
  cons = Consensus(profile)
  score = 0
  for motif in motifs: 
    for i in range(len(motif)): 
      if cons[i] != motif[i]:
        score+=1
  return score

def getRandom(n):
  return randint(0, n)

def FormRandomlyMotifs(genomes, k):
  motifs = []
  for genome in genomes:
    start = getRandom(len(genome)-k)
    motifs.append(genome[start:start+k])
  return motifs

def generateMotifsSet(profile_matrix, k, genomes):
  probKmers = []
  for genome in genomes:
    probKmers.append(ProfileMostProbableKmer(genome, k, profile_matrix))
  return probKmers

def RandomizedMotifSearch(genomes, k, t):
  motifs = FormRandomlyMotifs(genomes, k)
  bestMotifs = motifs
  while True:
    profile_matrix = ProfileMatrix(motifs)
    motifs = generateMotifsSet(profile_matrix, k, genomes)
    if Score(motifs) < Score(bestMotifs):
      bestMotifs = motifs
    else:
      return bestMotifs


k = 8
t = 5
with open('/content/rosalind_ba2f.txt', 'r') as f:
  genomelist = [line.strip() for line in f.readlines()]
  
best_score = float("Inf")
best_result = []
for i in range(1000):
  ans = RandomizedMotifSearch(genomelist, k, t)
  curr_score = Score(ans)
  if curr_score <= best_score:
    best_score = curr_score
    best_result = ans
for line in best_result:
  print(line)

TCTCGGGG
CCAAGGTG
TACAGGCG
TTCAGGTG
TCCACGTG


#### BA2G: Implement GibbsSampler

In [15]:
import numpy as np
import random
from random import randint, choices

def FormBestMotifs(genomes, k,t):
  bmotifs = []
  for genome in genomes:
    bmotifs.append(genome[:k])
  return bmotifs

def generateTextKmer(text, k):
  kmers = []
  for i in range(len(text)-k+1):
    kmers.append(text[i:i+k])
  return kmers


def FormCountMatrix(motifs, k): 
  count = np.zeros((4,len(motifs[0])))
  for motif in motifs:
    for i in range(len(motif)):
      if motif[i-1] == 'A':
        count[0][i-1] += 1
      elif motif[i-1] == 'C':
        count[1][i-1] += 1
      elif motif[i-1] == 'G':
        count[2][i-1] += 1
      elif motif[i-1] == 'T':
        count[3][i-1] += 1
  return count

def FormProfile(motifs, k): 
  countMat = FormCountMatrix(motifs, k)
  pseudoMat = PseudoCountMatrix(countMat, k)
  profileMatrx = np.zeros((4,len(motifs[0])))
  for row in range(4):
    for col in range(k):
      profileMatrx[row][col] = pseudoMat[row][col] / len(motifs)
  return profileMatrx

def PseudoCountMatrix(countMat, k): 
  pseudo = [1*4] * k   #np.ones((row, col)) here row = 4
  pseudoMatrix = pseudo + countMat
  return pseudoMatrix

  
def symbolToNumber(symbol):
	if symbol == "A":
		return 0
	if symbol == "C":
		return 1
	if symbol == "G":
		return 2
	if symbol == "T":
		return 3

def NumberToSymbol(number):
	if number == 0:
		return "A"
	if number == 1:
		return "C"
	if number == 2:
		return "G"
	if number == 3:
		return "T"

def Consensus(profile):
  consensus = ""
  for col in range(len(profile[0])):  
    max = 0
    loc = 0
    for row in range(4):
      if profile[row][col] > max:
        loc = row  
        max = profile[row][col]
    consensus += NumberToSymbol(loc)  
  return consensus


def Score(motifs, k):
  profile = FormProfile(motifs, k)
  cons = Consensus(profile)
  score = 0
  for motif in motifs:  
    for i in range(len(motif)): 
      if cons[i] != motif[i]:
        score+=1
  return score

def getRandom(n):
  return randint(0, n)

def FormRandomlyMotifs(genomes, k):
  motifs = []
  for genome in genomes:
    start = getRandom(len(genome)-k)
    motifs.append(genome[start:start+k])
  return motifs

def generateMotifsSet(profile_matrix, k, genomes):
  probKmers = []
  for genome in genomes:
    probKmers.append(ProfileMostProbableKmer(genome, k, profile_matrix))
  return probKmers

def probability(kmer, profile_matrix):
  prob = 1
  for j in range(len(kmer)):
      l = symbolToNumber(kmer[j])
      prob *= profile_matrix[l][j]
  return prob

def ProfileRandomGeneratedKmer(profile_matrix, k, genome):
  prob = []
  kmers = generateTextKmer(genome, k)
  for kmer in kmers:
    prob.append(probability(kmer, profile_matrix))
  
  motifs = random.choices(kmers, prob)
  return motifs[0]


def GibbsSampler(genomes, k, t, N):
  motifs = FormRandomlyMotifs(genomes, k)
  bestMotifs = motifs
  for j in range(N):
    i = getRandom(t-1)
    profile_matrix = FormProfile(motifs[:i] + motifs[i+1 :], k)
    motifi = ProfileRandomGeneratedKmer(profile_matrix, k, genomes[i])
    motifs[i] = motifi 
    if Score(motifs, k) < Score(bestMotifs, k):
      bestMotifs = motifs 
  return bestMotifs

k = 8
t = 5  
N = 100 
with open('/content/rosalind_ba2g.txt', 'r') as f:
  genomelist = [line.strip() for line in f.readlines()]

best = GibbsSampler(genomelist, k, t, N)
best_score = Score(best, k)
best_result = []
for i in range(20):
  ans = GibbsSampler(genomelist, k, t, N)
  curr_score = Score(ans, k)
  if curr_score < best_score:
    best_score = curr_score
    best_result = ans
for line in best_result:
    print(line)

CCCTCTCG
TAAGTGCC
CCGAGACC
CAGGTGCA
CACGTGCA


#### BA2H: Implement DistanceBetweenPatternAndStrings

In [16]:
def DistanceBetweenPatternsandGenome(genomelist, pattern):
  k = len(pattern)
  distance = 0
  for genome in genomelist:
    hammingDistance = float("inf")
    genome_kmers = generateTextKmer(genome, k)
    for kmer in  genome_kmers:
      if hammingDistance > HammingDistance(pattern, kmer):
        hammingDistance =  HammingDistance(pattern, kmer)
    distance = distance+hammingDistance
  #print(distance)  
  return distance

def generateTextKmer(text, k):
  kmers = set()
  for i in range(len(text)-k+1):
    kmers.add(text[i:i+k])
  return kmers

def HammingDistance(text1, text2):
    count=0;
    for i in range(len(text1)):
        if(text1[i] != text2[i]):
            count+=1
    return count


#pattern = input()
with open('/content/rosalind_ba2h.txt', 'r') as f:
  pattern = f.readline().strip()
  listWords = f.readline().strip().split(" ")
  #print(listWords)


print(DistanceBetweenPatternsandGenome(listWords, pattern))


5
